# Corpus Exploration Notebook

This notebook provides an interactive environment for exploring your atomized corpus.

## What you can do:
- Load and inspect the atomized corpus structure
- Browse themes, paragraphs, and sentences
- View statistics at each atomization level
- Search for specific content

## Setup

First, let's import the necessary modules and load the corpus.

In [ ]:
import sys
from pathlib import Path
import json

# Add framework to path
FRAMEWORK_ROOT = Path.cwd().parent.parent.parent
sys.path.insert(0, str(FRAMEWORK_ROOT))

# Import framework modules
from framework.core import Atomizer, Corpus, AtomLevel
from framework.core.ontology import Atom, Document

print(f"Framework root: {FRAMEWORK_ROOT}")
print("Framework modules loaded successfully!")

In [ ]:
# Configuration - Update these for your project
PROJECT_DIR = Path.cwd().parent
PROJECT_NAME = PROJECT_DIR.name

# Find atomized corpus file
RAW_DIR = PROJECT_DIR / "data" / "raw"
CORPUS_FILE = RAW_DIR / f"{PROJECT_NAME}_atomized.json"

print(f"Project: {PROJECT_NAME}")
print(f"Corpus file: {CORPUS_FILE}")
print(f"Exists: {CORPUS_FILE.exists()}")

## Load Corpus

In [ ]:
# Load the atomized corpus
if CORPUS_FILE.exists():
    corpus = Atomizer.load_json(CORPUS_FILE)
    print(f"Loaded corpus: {corpus.name}")
    print(f"Documents: {corpus.total_documents}")
else:
    print("ERROR: Corpus file not found!")
    print("Run atomization first: lingframe atomize -p <project-name>")

## Corpus Statistics

In [ ]:
# Count atoms at each level
stats = {
    "Themes": corpus.count_atoms(AtomLevel.THEME),
    "Paragraphs": corpus.count_atoms(AtomLevel.PARAGRAPH),
    "Sentences": corpus.count_atoms(AtomLevel.SENTENCE),
    "Words": corpus.count_atoms(AtomLevel.WORD),
    "Letters": corpus.count_atoms(AtomLevel.LETTER),
}

print("\n📊 Corpus Statistics")
print("=" * 40)
for level, count in stats.items():
    print(f"{level:15} {count:>10,}")

## Browse Themes

In [ ]:
# List all themes
def list_themes(corpus):
    """List all themes with their metadata."""
    themes = []
    for doc in corpus.documents:
        for theme in doc.root_atoms:
            themes.append({
                "id": theme.id,
                "title": theme.metadata.get("title", "(no title)"),
                "paragraphs": len([c for c in theme.children if c.level == AtomLevel.PARAGRAPH]),
                "preview": theme.text[:100] + "..." if len(theme.text) > 100 else theme.text,
            })
    return themes

themes = list_themes(corpus)
print(f"\n📖 Found {len(themes)} themes\n")

for i, theme in enumerate(themes[:10], 1):
    print(f"{i}. [{theme['id']}] {theme['title']}")
    print(f"   Paragraphs: {theme['paragraphs']}")
    print()

## Explore a Specific Theme

In [ ]:
def get_theme_by_index(corpus, index):
    """Get a theme by its index (0-based)."""
    count = 0
    for doc in corpus.documents:
        for theme in doc.root_atoms:
            if count == index:
                return theme
            count += 1
    return None

# Select theme to explore (change index as needed)
THEME_INDEX = 0
theme = get_theme_by_index(corpus, THEME_INDEX)

if theme:
    print(f"\n🔍 Theme: {theme.metadata.get('title', theme.id)}")
    print("=" * 60)
    print(f"ID: {theme.id}")
    print(f"\nContent preview:")
    print(theme.text[:500])
    print("...")

In [ ]:
# Show theme structure
if theme:
    print(f"\n📂 Theme Structure")
    print("=" * 60)
    
    for para in theme.children:
        if para.level == AtomLevel.PARAGRAPH:
            sentences = [c for c in para.children if c.level == AtomLevel.SENTENCE]
            print(f"\n  📄 {para.id} ({len(sentences)} sentences)")
            
            for sent in sentences[:3]:
                preview = sent.text[:60] + "..." if len(sent.text) > 60 else sent.text
                print(f"      → {preview}")

## Search Content

In [ ]:
def search_sentences(corpus, query, limit=10):
    """Search for sentences containing a query string."""
    results = []
    query_lower = query.lower()
    
    def search_recursive(atoms):
        for atom in atoms:
            if atom.level == AtomLevel.SENTENCE:
                if query_lower in atom.text.lower():
                    results.append({
                        "id": atom.id,
                        "text": atom.text,
                        "theme_id": atom.theme_id,
                    })
            search_recursive(atom.children)
    
    for doc in corpus.documents:
        search_recursive(doc.root_atoms)
    
    return results[:limit]

# Search for a term (modify as needed)
SEARCH_TERM = "example"  # Change this to your search term

results = search_sentences(corpus, SEARCH_TERM)
print(f"\n🔎 Search results for '{SEARCH_TERM}': {len(results)} found\n")

for i, result in enumerate(results, 1):
    print(f"{i}. [{result['id']}]")
    print(f"   {result['text'][:150]}..." if len(result['text']) > 150 else f"   {result['text']}")
    print()

## Word Frequency Analysis

In [ ]:
from collections import Counter
import re

def get_word_frequency(corpus, min_length=4, top_n=20):
    """Get word frequency across the corpus."""
    words = []
    
    def collect_words(atoms):
        for atom in atoms:
            if atom.level == AtomLevel.SENTENCE:
                # Extract words
                text_words = re.findall(r'\b[a-zA-Z]+\b', atom.text.lower())
                words.extend([w for w in text_words if len(w) >= min_length])
            collect_words(atom.children)
    
    for doc in corpus.documents:
        collect_words(doc.root_atoms)
    
    return Counter(words).most_common(top_n)

word_freq = get_word_frequency(corpus)
print("\n📊 Top 20 Words (4+ letters)\n")
for word, count in word_freq:
    bar = "█" * min(count // 5, 30)
    print(f"{word:15} {count:5} {bar}")

## Export for Further Analysis

In [ ]:
# Export corpus data as dict for custom analysis
corpus_dict = corpus.to_dict()
print(f"Corpus exported as dictionary with {len(corpus_dict)} top-level keys")
print(f"Keys: {list(corpus_dict.keys())}")

---

## Next Steps

Now that you've explored the corpus, you can:

1. **Run the evaluation analysis**: Open `evaluation.ipynb`
2. **Generate visualizations**: Open `visualization.ipynb`
3. **Run CLI commands**: `lingframe analyze -p <project-name>`